In [33]:
import pickle

with open(f"Data/rawdata", 'rb') as f:
    data = pickle.load(f)

with open(f"Data/uniqueBS", 'rb') as f:
    unique_bs = pickle.load(f)

In [34]:
class pc_potential:
    def __init__(self, line, id, parent_id, const_line, both_line, largeV_line):
        self.id = id
        self.constraint = line
        self.constc = const_line
        self.bothc = both_line
        self.largec = largeV_line
        self.potential = set()
        self.parent_id = parent_id
        self.child_list = []

class PathConditionTree:
    def __init__(self, unique_branchset):
        self.state_conditions = []
        self.state_tree = {-1:pc_potential(None, -1, None, None, None, None)}
        self.unique_branchset = unique_branchset

    def add_newdata(self, pc, bsidx, const_line, both_line, largeV_line):
        parent_id = -1
        for i, c in enumerate(pc):
            sc = pc[:i+1]
            try:
                scidx = self.state_conditions.index(sc)
            except:
                scidx = len(self.state_conditions)
                self.state_conditions.append(sc)
                self.state_tree[scidx] = pc_potential(c, scidx, parent_id, const_line[i], both_line[i], largeV_line[i])
                self.state_tree[parent_id].child_list.append(scidx)
            self.state_tree[scidx].potential |= self.unique_branchset[bsidx]
            parent_id = scidx
            

In [42]:
data['/home/jaehan/dd-klee/ours_experiments/minset_all_query_1/combine/result/0/10/test000002.ktest']

{'bsidx': 1,
 'path-condition': ['(Ult (ReadLSB w32 0 n_args) 2)',
  '(Slt 0 (ReadLSB w32 0 n_args))',
  '(Ult (ReadLSB w32 0 n_args_1) 3)',
  '(Slt 0 (ReadLSB w32 0 n_args_1))',
  '(Eq false (Eq 1 (ReadLSB w32 0 n_args_1)))',
  '(Eq false (Eq 0 (And w64 (ReadLSB w64 8 A-data-stat) 2147483647)))',
  '(Ult (ReadLSB w64 56 A-data-stat) 65536)',
  '(Eq false (Eq 0 (And w64 (ReadLSB w64 8 stdin-stat) 2147483647)))',
  '(Ult (ReadLSB w64 56 stdin-stat) 65536)',
  '(Eq false (Eq 0 (And w64 (ReadLSB w64 8 stdout-stat) 2147483647)))',
  '(Ult (ReadLSB w64 56 stdout-stat) 65536)',
  '(Eq 1 (ReadLSB w32 0 model_version))',
  '(Eq false (Eq 45 (Read w8 0 arg00)))',
  '(Eq 0 (Read w8 0 arg00))'],
 'const-abstract': ['( Ult ( ReadLSB w32 0 n_args ) 2 )',
  '( Slt 0 ( ReadLSB w32 0 n_args ) )',
  '( Ult ( ReadLSB w32 0 n_args_1 ) 3 )',
  '( Slt 0 ( ReadLSB w32 0 n_args_1 ) )',
  '( Eq false ( Eq 1 ( ReadLSB w32 0 n_args_1 ) ) )',
  '( Eq false ( Eq 0 ( And w64 ( ReadLSB w64 8 A-data-stat ) 214748364

In [36]:
tree_data = PathConditionTree(unique_bs)
for ktest in data:
    bsidx, pc, constpc, bothpc, lvpc = data[ktest].items()
    tree_data.add_newdata(pc[1], bsidx[1], constpc[1], bothpc[1], lvpc[1])

In [37]:
len(tree_data.state_tree)

2396

In [43]:
node_data = {}
for node_id, node_info in tree_data.state_tree.items():
    node_data[node_id] = {}
    node_data[node_id]["constraint"] = node_info.constraint
    node_data[node_id]["const_constraint"] = node_info.constc
    node_data[node_id]["both_constraint"] = node_info.bothc
    node_data[node_id]["large_constraint"] = node_info.largec
    node_data[node_id]["children"] = node_info.child_list
    node_data[node_id]["potential"] = len(node_info.potential)

In [51]:
import json
del node_data[-1]
with open(f"nodesData.json", 'w') as f:
    json.dump(node_data, f, indent=4)